In [1]:
# !source AI4INDUSTRY-env/bin/activate

In [2]:
# !pip install odfpy
# !pip install pandas
# !pip install openpyxl
# !pip install requests
# !pip install seaborn
# !pip install folium
# !pip install matplotlib
# !pip install scikit-learn

In [3]:
import pandas as pd

In [10]:
df2 = pd.read_csv('data_cleaned.csv', index_col=0)
df2.head(5)

,boutique,code_postal,client_anciennete,frequence_visites,captation,causes_achats,produit_prefere,produits_moins_connus,client_internet,volonte_boutique_proche,lat,lon,lat_boutique,lon_boutique
0,Saint-Jean-de-Luz,31410,Depuis quelques annees,exceptionelle,bouche_a_oreille,pour me faire plaisir; pour regaler ma famille,Gateau basque,Je ne sais pas,Non,Non,43.367108,1.272896,43.387076,-1.663569
1,Saint-Jean-de-Luz,64500,Depuis toujours,exceptionelle,bouche_a_oreille,pour me faire plaisir; pour regaler ma famille...,Autres,Autres,Non,sans_avis,43.388867,-1.638179,43.387076,-1.663569
2,Saint-Jean-de-Luz,62400,Depuis quelques annees,regulier,boutique,pour me faire plaisir,Autres,Autres,Non,sans_avis,50.528190,2.642115,43.387076,-1.663569
3,Saint-Jean-de-Luz,33820,Depuis quelques annees,exceptionelle,boutique,pour me faire plaisir; pour faire un cadeau,Autres,Autres,Non,oui,61.466562,23.819322,43.387076,-1.663569
4,Saint-Jean-de-Luz,64210,Depuis quelques annees,exceptionelle,bouche_a_oreille_et_boutique,pour me faire plaisir; pour faire un cadeau,Autres,Gateau basque aux agrumes,Non,sans_avis,43.437635,-1.585311,43.387076,-1.663569


In [ ]:
df2['code postal'] = df2['code postal'].astype(str).str.replace('*', '0')

/tmp/ipykernel_4141/1107489302.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['code postal'] = df2['code postal'].astype(str).str.replace('*', '0')


In [ ]:
# Créer une liste de code postaux uniques
postal_list = list(set(df2['code postal'][df2['code postal'].astype(str).str.match(r'^\d{5}$')]))
postal_list = set(df2['code postal'].value_counts().keys().to_list())
postal_list[0:5]

TypeError: 'set' object is not subscriptable

# Get lon and lat

In [ ]:

# import requests

# def set_lon_lat(df, postal_list):
#     df['lat'] = ""
#     df['lon'] = ""
    
#     responses_dic = {i:requests.get(f'https://nominatim.openstreetmap.org/search?q={str(i)}&format=json') for i in postal_list}
#     for index, row in df.iterrows():
#         df.loc[index, 'lat'] = responses_dic[df.loc[index, 'code postal']].json()[0]['lat']
#         df.loc[index, 'lon'] = responses_dic[df.loc[index, 'code postal']].json()[0]['lon'] 
#     df.to_csv('data_with_coordinates.csv', index=False)
# set_lon_lat(df2, postal_list)

In [ ]:
#Last cleaning
import pandas as pd
df2 = pd.read_csv('data_cleaned.csv', index_col=0)
df2 = df2.replace({'é': 'e', 'è': 'e', 'ê': 'e', 'ë': 'e'}, regex=True)
df2 = df2.replace({'â': 'a', 'à': 'a'}, regex=True)
df2 = df2.replace({'û': 'u', 'ù': 'u', 'ü': 'u'}, regex=True)
df2 = df2.replace({'ô': 'o'}, regex=True)
df2['frequence_visites'] = df2['frequence_visites'].replace({'visite mensuelle': 'regulier','3 fois par semaine': 'regulier','visite hebdomadaire': 'regulier','Tous les 15 jours': 'regulier','Tous les jours': 'regulier','Tous les 2 jours': 'regulier','visite hebdomadaire; visite exceptionnelle': 'regulier',"C'est la premiere fois !": 'premiere_fois','Par trimetres': 'exceptionelle',"Surtout l'ete": 'exceptionelle',"Surtout l’ete": 'exceptionelle','Par an': 'exceptionelle','Surtout en ete': 'exceptionelle',"Surtout l'ete; Noel": 'exceptionelle','visite exceptionnelle; Surtout en ete': 'exceptionelle', 'visite exceptionnelle, Surtout en ete': 'exceptionelle','visite exceptionnelle': 'exceptionelle'})
df2['produit_prefere'].fillna('sans_avis', inplace=True)
df2['produits_moins_connus'].fillna('pas_repondu', inplace=True)
df2['volonte_boutique_proche'].fillna('sans_avis', inplace=True)
counts = df2['produit_prefere'].value_counts()
top_5 = counts.head(5).index.tolist()
df2['produit_prefere'] = df2['produit_prefere'].apply(lambda x: x if x in top_5 else 'Autres')
df2['produits_moins_connus'] = df2['produits_moins_connus'] .replace({'pas_repondu': 'Je ne sais pas'})
counts2 = df2['produits_moins_connus'].value_counts()
least_5 = counts2.head(5).index.tolist()
df2['produits_moins_connus'] = df2['produits_moins_connus'].apply(lambda x: x if x in least_5 else 'Autres')#df2

In [ ]:
#GET lat and lon from boutiques
import requests
import pandas as pd

df = pd.read_csv('data_cleaned.csv', index_col=0).replace('St Jean', 'Saint-Jean-de-Luz')
boutiques = df.boutique.value_counts().keys().to_list()
def set_lon_lat(df, list):
    df['lat_boutique'] = ""
    df['lon_boutique'] = ""
    
    responses_dic = {i:requests.get(f'https://nominatim.openstreetmap.org/search?q={str(i)}&format=json') for i in list}
    for index, row in df.iterrows():
        df.loc[index, 'lat_boutique'] = responses_dic[df.loc[index, 'boutique']].json()[0]['lat']
        df.loc[index, 'lon_boutique'] = responses_dic[df.loc[index, 'boutique']].json()[0]['lon'] 
    df.to_csv('data_with_coordinates.csv', index=False)

set_lon_lat(df, boutiques)


In [7]:
l = ["par hasard en passant devant","via les reseaux","par hasard en passant devant; bouche a oreille"]
def cleanning_captation(x) :
    if x in l :
        if x == l[0]:
            x = "boutique"
        elif x == l[1] :
            x = "reseaux"
        elif x == l[2] :
            x = "bouche_a_oreille_et_boutique"
    else :
        x = "bouche_a_oreille"
    return x
df2["captation"] = df2["captation"].apply(cleanning_captation)
df2["captation"].value_counts()